In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import sys

import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)


import seaborn as sns
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout
# from tensorflow.keras.callbacks import ModelCheckpoint

In [26]:
cg = {'a','b','c','d'}
OHEs = {}
Scalers = {}
for x in cg:
    Scalers[x] = StandardScaler()
    OHEs[x] = OneHotEncoder()

In [27]:
TTS_DICT = {
        'a':['50','50'],
        'b':['60',"40"],
        'c':['75','25'],
        'd':['80','20']
    }

TTS_DICT_INV = {
        '50':'a',
        '40':'b',
        '25':'c',
        '20':'d'    
    }

print(TTS_DICT_INV)

TTS_BASE_DIR = os.path.abspath('Train_Test_Data_Splits')
TTS_FILE_LIST = os.listdir(TTS_BASE_DIR)
print(TTS_FILE_LIST)

{'50': 'a', '40': 'b', '25': 'c', '20': 'd'}
['tts_test_50_50.csv', 'tts_test_60_40.csv', 'tts_test_75_25.csv', 'tts_test_80_20.csv', 'tts_train_50_50.csv', 'tts_train_60_40.csv', 'tts_train_75_25.csv', 'tts_train_80_20.csv']


In [28]:
x_trains = {}
y_trains= {}
x_tests = {}
y_tests = {}

for tts_file in TTS_FILE_LIST:
    tts_file_path = os.path.join(TTS_BASE_DIR,tts_file)
    df = pd.read_csv(tts_file_path,index_col=False)
    tts_file_wo_ext = tts_file.split('.')[0].split('_')
    k = TTS_DICT_INV[tts_file_wo_ext[-1]]
    if 'train' in tts_file_wo_ext:
        #depend on test set size
        x_trains[k]  = Scalers[k].fit_transform(df.iloc[:,:-1].values)
        y_trains[k]  = OHEs[k].fit_transform(df['Y_train'].values.reshape(-1,1)).toarray()
    else:
        x_tests[k]  = Scalers[k].fit_transform(df.iloc[:,:-1].values)
        y_tests[k]  = OHEs[k].fit_transform( df['Y_test'].values.reshape(-1,1)).toarray()

print(x_trains['b'].shape)      
print(y_trains['b'].shape)      
print(y_tests['b'].shape)      
print(x_tests['b'].shape)      
        

(2592, 162)
(2592, 8)
(1728, 8)
(1728, 162)


---
## Model 1
----

In [22]:
def model_1():
    model=Sequential()
    model.add(Conv1D(256, kernel_size=3, strides=1, padding='same', activation='relu', input_shape=(x_train.shape[1], 1)))
    model.add(MaxPooling1D(pool_size=3, strides = 1, padding = 'same'))

    model.add(Conv1D(128, kernel_size=5, strides=2, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))
    model.add(Dropout(0.25))


    model.add(Flatten())
    model.add(Dense(units=32, activation='relu'))
    model.add(Dropout(0.25))


    model.add(Dense(units=8, activation='softmax'))
    model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])
    return model

In [23]:
model = model_1()

NameError: name 'x_train' is not defined

In [ ]:
def loss_accuracy_details(model,epoch,x_test,y_test):
    print("Accuracy of our model on test data : " , model.evaluate(x_test,y_test)[1]*100 , "%")

    history  = model.hi
    
    epochs = [i for i in range(epoch)]
    fig , ax = plt.subplots(1,2)
    train_acc = history.history['accuracy']
    train_loss = history.history['loss']
    test_acc = history.history['val_accuracy']
    test_loss = history.history['val_loss']

    fig.set_size_inches(20,6)
    ax[0].plot(epochs , train_loss , label = 'Training Loss')
    ax[0].plot(epochs , test_loss , label = 'Testing Loss')
    ax[0].set_title('Training & Testing Loss')
    ax[0].legend()
    ax[0].set_xlabel("Epochs")

    ax[1].plot(epochs , train_acc , label = 'Training Accuracy')
    ax[1].plot(epochs , test_acc , label = 'Testing Accuracy')
    ax[1].set_title('Training & Testing Accuracy')
    ax[1].legend()
    ax[1].set_xlabel("Epochs")
    plt.show()